In [ ]:
#playing around with multisets to spread out weight of each profile over events/nodes


In [1]:
import datasketch 
import numpy as np
import time


In [2]:
from datasketch import WeightedMinHashGenerator

v1 = 1000 *[.5]
v2 = 1000 * [1]

# WeightedMinHashGenerator requires dimension as the first argument
wmg = WeightedMinHashGenerator(1000, sample_size=512, seed=0)

wm1 = wmg.minhash(v1) # wm1 is of the type WeightedMinHash
wm2 = wmg.minhash(v2)

print("Estimated Jaccard is", wm1.jaccard(wm2))



Estimated Jaccard is 0.51171875


In [3]:
#each hog has 4000 taxonomic elements
#total weight is defined as 1 

#spread out the hog over its taxonomic range 
#give more weight to certain events/levels

#score over the HOG -> normalize

import numpy as np




In [4]:
mastertree = Tree( 'mastertree.nwk' , format = 1 )  
print(len(mastertree))
treeindex = { n.name : i for i,n in enumerate(mastertree.traverse())}
print(len(treeindex))

2103
2997


In [5]:
print(len(descendants))

7150


In [6]:
#assign a weight to each taxonomic level. decrese exponentially or linearly?
#exp should be tuned for each event type

#todo sep functions for 3 kingdoms... 
exp_bact = .9
exp_euk = .9
exp_arch = .9


for node in mastertree.traverse():
    node.add_feature('degree', 1 )
for node in mastertree.iter_descendants():
    for d in node.iter_descendants():
        d.degree+=1
degreedict = {n.name : n.degree for n in mastertree.traverse()}        
print(degreedict)

{'': 1, '290512': 5, '591167': 7, '1350': 7, '359160': 10, '5543': 13, '553199': 8, '930943': 8, '1163389': 6, '885275': 8, '256319': 5, '302911': 8, '7165': 17, '530564': 6, '523850': 5, '222891': 7, '349519': 8, '51351': 13, '510516': 12, '469': 6, '929562': 7, '114380': 5, '708248': 7, '60480': 6, '1453429': 7, '1773': 9, '5053': 12, '290317': 8, '1002672': 5, '402612': 8, '1679': 8, '218494': 9, '709986': 6, '227941': 5, '34073': 6, '525909': 4, '479436': 5, '1508657': 7, '749219': 6, '314145': 17, '940557': 4, '61853': 23, '154': 6, '123214': 5, '926562': 6, '562970': 7, '226185': 9, '54388': 9, '1150': 5, '155864': 9, '85030': 5, '85025': 6, '85643': 6, '349101': 8, '188711': 2, '633': 8, '395491': 10, '4792': 6, '5758': 3, '585395': 8, '234826': 8, '1129': 6, '188786': 4, '699246': 7, '262768': 7, '517418': 5, '953739': 7, '401053': 5, '189423': 9, '552536': 8, '49546': 6, '9305': 16, '2419': 3, '1613': 8, '119857': 7, '634498': 6, '85006': 5, '537457': 7, '585055': 8, '1091': 6

In [7]:
import pyham

ortho = ''
treestr = mastertree.write(format = 1)




In [ ]:

#add an exponential bleed function for events gain, dup, loss
#bleedup, bleeddown should be diff for each event type

bleedup = .1
bleeddown = .1
#assign score to nodes

#traverse tree profile and calculate scores
for n in treeprofile.traverse():
    #add score for an event based on degree
    n.add_feature('score', exp_bact**degreedict[n.name])

#blur scores up and down
for e in treeprofile.traverse():
    if e.score>0:
        init = e.score
        for n in e.iter_ancestors():
            if n.score:
                n.score *= bleedup
            else:
                n.score = bleedup

        for n in e.iter_descendants():
            if n.score:
                n.score *= bleeddown
            else:
                n.score = bleeddown

#after a treeprofile is distributed to it taxonomic nodes, normalize
vector = np.asarray([ n.score for n in treeprofile.traverse() ]) 
vector /= np.sum(vector)
#turn into weighted minhash

